In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim

%load_ext autoreload
%autoreload 2

In [22]:
#from forward import *
import sys
sys.path.append("../static/")
sys.path.append("..")
import kinematics
from inverse import with_torch
from presets_robot_models import preset_models
#from learn import evaluate_plot, train_motors
from learn import *

In [3]:
alpha, theta, radius, dists, active, limits = get_DH_params(preset_models["HangingArm"])

In [4]:
For_model = kinematics.Kinematics("HangingArm")

In [8]:
model = torch.load("./kine_models/HangingArm.pt")

# For Training

In [14]:
For_model.orientation = False
inputs, target = For_model.generate_maps()
n_in  = inputs.shape[1]
n_out = target.shape[1]
model = NeuralNetworkStack(n_in, n_out, nlayers=5, depth=20)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# generate map

In [9]:
inputs, target = For_model.generate_maps()

In [10]:
For_model.orientation = False

In [39]:
evaluate_plot(For_model, model)

tensor(19915.1348, grad_fn=<MeanBackward0>)


In [40]:
train_positions(For_model, model)

KeyboardInterrupt: 

In [27]:
train_motors(For_model, model)

15508.1734375
15311.9310546875
15493.7333984375
15393.99296875
15633.20322265625
15465.96279296875
15479.7365234375
15717.01328125
15545.95546875
15464.4021484375
15236.5833984375
15491.66689453125
15595.97353515625
15351.05380859375
15398.5044921875
15582.6048828125
15475.650390625
15305.2451171875
15562.95986328125
15369.44208984375
15595.99736328125
15534.0830078125
15475.1322265625
15511.1529296875
15288.4583984375
15351.25009765625
15248.04931640625
15489.35166015625
15354.11513671875
15399.0076171875
15414.49267578125
15355.9619140625
15398.538671875
15562.9498046875
15592.963671875
15544.88935546875
15429.11650390625
15447.11064453125
15432.9912109375
15386.6359375
15450.99130859375
15329.93955078125
15483.75703125
15534.056640625
15447.6134765625
15423.1640625
15338.5029296875
15378.3140625
15523.3537109375
15284.28486328125
15588.59130859375
15588.1798828125
15326.607421875
15566.92998046875
15422.88173828125
15644.795703125
15232.05703125
15281.10771484375
15402.40986328125
1

# Prototype code to test Machine

In [18]:
inputs, target = For_model.generate_maps(n=1)
#mot_pos = np.array([alpha,theta,radius,dists])
n= np.random.randint(len(inputs))
#######################
pred = model(torch.Tensor(inputs[n:n+1])).detach()
######################
For_model.set_active(pred[0])
For_model.update_position()
pos_pred = For_model.run_forward().round(2)
#######################
For_model.set_active(target[n])
pos_targ = For_model.run_forward().round(2)
########################
print("--- x y z ---")
print(inputs[n].round(2))
print(pos_targ)
print(pos_pred )
print("--- torch ---")
pred_xys = with_torch().forward_from_active(For_model, target[n:n+1])
print(pred_xys[0,0].numpy().round(2))
pred_xys = with_torch().forward_from_active(For_model, pred)
print(pred_xys[0,0].numpy().round(2))
print("--- motor positions ---")
print(target[n].round(2))
print(pred.numpy().round(2)[0])

--- x y z ---
[ 38.22  23.27 190.01]
[ 38.22  23.27 190.01]
[ 36.03  23.03 189.73]
--- torch ---
[ 38.22  23.27 190.01]
[ 37.12  24.02 190.29]
--- motor positions ---
[ 14  22 267 129 156]
[ 65.53  21.72  94.63 127.97 150.83]


In [36]:
def get_DH(For_model):
    alpha  = torch.tensor(1.*np.array(For_model.alpha)) 
    theta  = torch.tensor(1.*np.array(For_model.theta)) 
    radius = torch.tensor(1.*np.array(For_model.radius))
    dists  = torch.tensor(1.*np.array(For_model.dists)) 
    active = For_model.active
    DH = torch.stack([alpha, theta, radius, dists])
    return DH

def get_transform(DH):
    alpha, theta, radius, dists = DH
    alpha = torch.deg2rad(alpha)
    theta = torch.deg2rad(theta)
    
    TFs = make_transforms(alpha, theta, radius, dists)
    return TFs[-1]

def make_transforms(alpha, theta, radius, dists):

    mot_params = torch.stack([alpha, theta, radius, dists])

    Tfs = []
    T = torch.eye(4)
    for i in range(mot_params.shape[1]):
        a,t,r,d = mot_params[:,i]
        DHt = DH_transform(a,t,r,d) 
        T = torch.matmul(T,DHt)     
        Tfs.append(T)

    return Tfs

def DH_transform(a,t,r,z):
        Zt = Z_transform(a,t,r,z)
        Xt = X_transform(a,t,r,z)
        DHt = torch.matmul(Zt,Xt)
        return DHt

    ###############################################
def X_transform(a,t,r,d):

    Xt = torch.eye(4)
    Xt[0,3] = r
    Xt[1,1] = torch.cos(a)
    Xt[1,2] = -1*torch.sin(a)
    Xt[2,1] = torch.sin(a)
    Xt[2,2] = torch.cos(a)

    return Xt

def Z_transform(a,t,r,d):

    Zt = torch.eye(4)
    Zt[0,0] = torch.cos(t)
    Zt[0,1] = -1*torch.sin(t)
    Zt[1,0] = torch.sin(t)
    Zt[1,1] = torch.cos(t)
    Zt[2,3] = d

    return Zt

In [37]:
matrix = torch.Tensor(np.eye(3))

In [327]:
rot_angle = torch_matrix_to_rotation( matrix )
rot_angle

tensor([1.5708, 1.0472, 3.1416])

In [303]:
DH = get_DH(For_model)
TF = get_transform(DH)
matrix = TF[:3,:3]

In [304]:
rotation = np.array(rotation)

tensor(-0.1879)

In [326]:
DH = get_DH(For_model)
TF = get_transform(DH)
matrix = TF[:3,:3]

matrix = torch.Tensor(R.from_euler("XYZ",[np.pi/2,np.pi/3,np.pi]).as_matrix())

In [346]:
for _ in range(100):
    rot = np.random.uniform(np.pi,size=(3))
    out = R.from_matrix(R.from_euler("XYZ",rot).as_matrix()).as_euler("XYZ")
    if out[0]<0:  
        out[[0,2]] = out[[0,2]] + np.pi
        out[[1]]   = np.pi - out[[1]] 
    print(rot, out, rot.round(4) == out.round(4)) 

[2.48298614 1.06484103 1.49274094] [2.48298614 1.06484103 1.49274094] [ True  True  True]
[2.84290397 2.00141479 1.03128723] [2.84290397 2.00141479 1.03128723] [ True  True  True]
[1.72917101 1.89129523 2.81905332] [1.72917101 1.89129523 2.81905332] [ True  True  True]
[1.13833074 2.71066973 2.90375832] [1.13833074 2.71066973 2.90375832] [ True  True  True]
[2.60491339 2.06040844 1.83190064] [2.60491339 2.06040844 1.83190064] [ True  True  True]
[2.43085511 2.44070251 2.99944293] [2.43085511 2.44070251 2.99944293] [ True  True  True]
[2.65715894 1.17668211 1.53203542] [2.65715894 1.17668211 1.53203542] [ True  True  True]
[1.35047719 1.17232089 3.03529087] [1.35047719 1.17232089 3.03529087] [ True  True  True]
[1.60379403 1.88811651 1.35566381] [1.60379403 1.88811651 1.35566381] [ True  True  True]
[2.12680618 3.01411359 2.46566624] [2.12680618 3.01411359 2.46566624] [ True  True  True]
[3.08487961 1.19153305 2.62116847] [3.08487961 1.19153305 2.62116847] [ True  True  True]
[1.0865435

In [ ]:
For_model = kinematics(alpha, theta, radius, dists, active, limits)

In [216]:
inputs, target = For_model.generate_maps()
#mot_pos = np.array([alpha,theta,radius,dists])
#n= np.random.randint(len(inputs))
pred = model(torch.Tensor(inputs)).detach().numpy()

In [ ]:
For_model = kinematics(alpha, theta, radius, dists, active, limits)

In [221]:
For_model.orientation = True

## 2D TEST

In [9]:
size = 1000
t = np.random.randint(-10,370,size=size)
r = np.random.randint(100,size=size)
inputs = np.array([r,t]).T

x = np.sin(np.radians(t))*r
y = np.cos(np.radians(t))*r
target = np.array([x,y]).T

inputs,target = target,inputs

In [10]:
n_in  = inputs.shape[1]
n_out = target.shape[1]
model = NeuralNetworkStack(n_in, n_out, nlayers=15, depth=100)
#model = NeuralNetworkSinusoid(n_in, n_out)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
#optimizer = optim.Adam(model.parameters(), lr=1e-3)
for n in range(100):
    size = 500
    t = np.random.randint(-10,370,size=size)
    r = np.random.randint(150,size=size)
    inputs = np.array([r,t]).T

    x = np.sin(np.radians(t))*r
    y = np.cos(np.radians(t))*r
    target = np.array([x,y]).T
    
    train_loop(model, inputs, target, distance_loss,optimizer)
        
    if n % 10 == 0:
        optimizer.param_groups[0]["lr"] = optimizer.param_groups[0]["lr"]*0.5        

50.13558605194092
41.69785896301269
27.24435827255249
20.57340579986572
14.285087652206421
11.445655641555787
10.45282082080841
10.098252668380738
7.19429224729538
6.440871031284332
6.51929044008255
2.7458794486522673
3.3659899294376374
3.504760512113571
3.526837866306305
3.1385426807403562
3.0256880462169646
2.843242418766022
2.5200929391384124
2.5201621651649475
2.930586656332016
1.8759654599428177
1.187305451631546
2.156197493672371


KeyboardInterrupt: 

In [300]:
n= np.random.randint(len(inputs))
xy = target[n]
pred = model(torch.Tensor(inputs[n:n+1])).detach().numpy()
print(inputs[n])
print(target[n])
print(pred.round(2))

[ 43 163]
[ 12.5719833  -41.12110451]
[[ 12.65 -41.17]]


In [16]:
size = 1000
t = np.random.randint(-10,370,size=size)
r = np.random.randint(150,size=size)
inputs = np.array([r,t]).T

x = np.sin(np.radians(t))*r
y = np.cos(np.radians(t))*r
target = np.array([x,y]).T

inputs, target = target, inputs

pred = model(torch.Tensor(inputs)).detach().numpy()



r = inputs[:,0]
t = inputs[:,1]

x = target[:,0]
y = target[:,1]

x1 = r*np.sin(np.radians(t))
y1 = r*np.cos(np.radians(t))

x2 = pred[:,0]
y2 = pred[:,1]

x2 = r*np.sin(np.radians(x2))
y2 = r*np.cos(np.radians(y2))

plt.plot(x,y,"bo")
#plt.plot(x1,y1,)
plt.plot(x2,y2,"ro")
plt.plot([x,x2],[y,y2],"g")

In [13]:
size = 1000

pred = model(torch.Tensor(inputs)).detach().numpy()

x = target[:,0]
y = target[:,1]

x2 = pred[:,0]
y2 = pred[:,1]


plt.plot(x,y,"bo")
#plt.plot(x1,y1,)
plt.plot(x2,y2,"ro")
plt.plot([x,x2],[y,y2],"g")

In [522]:
    def forward(self,x_in):
                
        x_next = self.layer_first(x_in)
    
        out_list = [x_in,x_next]
        for n, block in enumerate(self.blocks):
            x_next = block(x_next) 
            out_list.append(x_next)  
            
        x = torch.cat(out_list, dim=1)
        
        x = F.leaky_relu(x)
        print(x)

        out = self.layer_last(x)
        
        return out